In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Tuple, List, Callable, Literal
import os
import shutil

## Generowanie wartości x

In [ ]:
def generate_chebyshev_roots(n: int, a: np.float64, b: np.float64):
  # adjust n to account for the two additional nodes at the ends
  n_adjusted = n - 2
  chebyshev_roots = np.array([(a + b) / 2 + (b - a) / 2 * np.cos((2 * k + 1) / (2 * n_adjusted) * np.pi) for k in range(n_adjusted)], dtype=np.float64)
  # add the endpoints of the interval
  return np.concatenate(([a], chebyshev_roots[::-1], [b]))

def generate_function_chebyshev_nodes(n: int, interval: tuple[np.float64, np.float64], func):
  a, b = interval[0], interval[1]
  xs = generate_chebyshev_roots(n, a, b)
  tups = []
  for x in xs:
    tups.append((x, func(x)))
  return np.array(tups)


def generate_function_uniform_nodes(n: int, interval: tuple[np.float64, np.float64], func):
  a,b= interval[0], interval[1]
  xs = np.linspace(a,b,n)
  tups = []
  for x in xs:
    tups.append((x,func(x)))
  return np.array(tups)


# Interpolacja Funkcją Sklejaną Drugiego Stopnia

In [ ]:
def get_quadratic_spline_interpolation(nodes: list[list[np.float64, np.float64]], func_derivative: Callable, boundary_condition: str = "free") -> Tuple[List[float], List[float], List[float]]:
  xs, ys = nodes[:,0], nodes[:, 1]
  n = len(xs)  # nodes number
  
  # coefficients lists
  a = [0.0] * (n - 1)
  b = [0.0] * n
  c = [0.0] * (n - 1)
  
  # h_i = x_(i+1) - x_i
  h = [xs[i+1] - xs[i] for i in range(n-1)]
  
  # set of equations init
  A = np.zeros((n, n))
  B = np.zeros(n)
  
  # boundary conditions
  if boundary_condition == "free":
    # b_1 = 0
    A[0, 0] = 1.0
    B[0] = 0.0
  elif boundary_condition == "clamped":
    # second derivative clamped boundary
    # -b_1 + b_2 = 1/2 * f''(x_1) + h_i
    deriv_x1 = func_derivative(xs[0])
    
    A[0, 0] = -1.0
    A[0, 1] = 1.0
    B[0] = 0.5 * deriv_x1 * h[0]
  else:
    raise ValueError("Unknown boundary condition. Use 'free' or 'clamped'.")
  
  # fill the remaining coefficients in set of equations
  for i in range(1, n):
    A[i, i-1] = 1.0
    A[i, i] = 1.0
    B[i] = 2 * (ys[i] - ys[i-1]) / h[i-1]
  
  # solve
  # print(A)
  b = np.linalg.solve(A, B)
  
  # rest of coefficients
  for i in range(n-1):
    c[i] = ys[i]
    a[i] = (b[i+1] - b[i]) / (2 * h[i])
  
  def f(x):
    i = 0
    while i < n-1 and x > xs[i+1]:
      i += 1
    
    if i == n-1:
      i = n-2
    return a[i] * (x - xs[i])**2 + b[i] * (x - xs[i]) + c[i]

  return f

# Interpolacja Funkcja Sklejaną Trzeciego Stopnia

In [ ]:
def get_cubic_spline_interpolation(nodes, boundary_condition: Literal["free", "not-a-knot"] = "free",) -> Tuple[List[float], List[float], List[float], List[float]]:
  xs, ys = nodes[:,0], nodes[:,1]
  n = len(xs)  # nodes number
  
  h = [xs[i+1] - xs[i] for i in range(n-1)]
  
  A = np.zeros((n, n))
  b = np.zeros(n)
  
  for i in range(1, n-1):
    A[i, i-1] = h[i-1]
    A[i, i] = 2 * (h[i-1] + h[i])
    A[i, i+1] = h[i]
    
    b[i] = 6 * ((ys[i+1] - ys[i]) / h[i] - (ys[i] - ys[i-1]) / h[i-1])
  
  if boundary_condition == "free":
      # S''(x_0) = S''(x_{n-1}) = 0
      A[0, 0] = 1.0
      A[n-1, n-1] = 1.0
      b[0] = 0.0
      b[n-1] = 0.0
      
  elif boundary_condition == "not-a-knot":
    # S'''_0(x_1) = S'''_1(x_1) i S'''_{n-3}(x_{n-2}) = S'''_{n-2}(x_{n-2})
    if n > 3:  # requires minimum 4 points to be able to apply this
      A[0, 0] = h[1]
      A[0, 1] = -(h[0] + h[1])
      A[0, 2] = h[0]
      b[0] = 0.0
      
      A[n-1, n-3] = h[n-2]
      A[n-1, n-2] = -(h[n-3] + h[n-2])
      A[n-1, n-1] = h[n-3]
      b[n-1] = 0.0
    else:
      # free if not enough points
      A[0, 0] = 1.0
      A[n-1, n-1] = 1.0
      b[0] = 0.0
      b[n-1] = 0.0
  else:
    raise ValueError("Unknown boundary condition. Use 'free' or 'not-a-knot'.")
  
  # solve
  M = np.linalg.solve(A, b)
  

  a = [0.0] * (n-1)
  b = [0.0] * (n-1)
  c = [0.0] * (n-1)
  d = [0.0] * (n-1)
  
  for i in range(n-1):
    a[i] = (M[i+1] - M[i]) / (6 * h[i])
    b[i] = M[i] / 2
    c[i] = (ys[i+1] - ys[i]) / h[i] - h[i] * (M[i+1] + 2 * M[i]) / 6
    d[i] = ys[i]
      
  def f(x):
    i = 0
    while i < n-1 and x > xs[i+1]:
      i += 1
    
    if i == n-1:
      i = n-2
    return a[i] * (x - xs[i])**3 + b[i] * (x - xs[i])**2 + c[i] * (x-xs[i]) + d[i]

  return f

## Błąd Pomiarowy

In [ ]:
NUMBER_OF_PROBES = 1000

def get_max_error(func, interpolation_func, interval):
  xs = np.linspace(interval[0], interval[1], NUMBER_OF_PROBES)
  return max([np.abs(func(x) - interpolation_func(x)) for x in xs])

def get_squared_error(func, interpolation_func, interval):
  xs = np.linspace(interval[0], interval[1], NUMBER_OF_PROBES)
  return np.sqrt(sum([(func(x) - interpolation_func(x))**2 for x in xs])) / NUMBER_OF_PROBES

# Wizualizacja Funkcji

In [ ]:
def plot_function(
  x_values: np.ndarray[np.float64], 
  func: Callable[[np.float64], np.float64], 
  nodes: np.ndarray[(np.float64, np.float64)] | None = None,
  x_lim: tuple[float, float] | None = None,
  y_lim: tuple[float, float] | None = None,
  x_scale: str = 'linear',
  y_scale: str = 'linear',
  function_name: str = 'Funkcja bazowa',
  nodes_name: str = 'Węzły interpolacji',
  color: str | None = None
):
  y_values = np.array([func(x) for x in x_values])

  if color is not None:
    plt.plot(x_values, y_values, label=function_name, color=color)
  else:
    plt.plot(x_values, y_values, label=function_name)
    
  if nodes is not None:
    plt.scatter(nodes[:, 0], nodes[:, 1], color='red', label=nodes_name)
  
  plt.xscale(x_scale)
  plt.yscale(y_scale)
  
  if x_lim is not None:
    plt.xlim(x_lim)
  if y_lim is not None:
    plt.ylim(y_lim)
    
  # Dodanie siatki
  plt.grid(True)
  
  plt.xlabel('x')
  plt.ylabel('y')
  plt.legend()
  plt.title('Wizualizacja funkcji')


In [ ]:
m = 5
k = 0.5
interval = (-5, 5)
N_max = 25 # Maximum number of nodes to test

func = lambda x: x**2 - m*np.cos((np.pi * x) / k)
# Original implementation of clamped quadratic used the second derivative
func_second_deriv = lambda x: 2 + ((m * np.pi**2) / k**2) * np.cos((np.pi * x) / k) 
N_SET = [5,7,10,13,20,50,100,1000]

# Wartości

In [ ]:
N_max=150
PATH_TO_SAVE_MAIN = os.path.join('.', 'img_splines_only_with_clamped') # Updated path
PATH_TO_SAVE_DATA = os.path.join('.', 'data_splines_only_with_clamped') # Updated path

# Cleanup
if os.path.exists(PATH_TO_SAVE_MAIN):
  shutil.rmtree(PATH_TO_SAVE_MAIN)
if os.path.exists(PATH_TO_SAVE_DATA):
  shutil.rmtree(PATH_TO_SAVE_DATA)

os.makedirs(PATH_TO_SAVE_DATA, exist_ok=True)

paths = {
  'spline_quadratic_free': {'linear': os.path.join(PATH_TO_SAVE_MAIN, 'spline_quadratic_free', 'linear'),
                            'chebyshev': os.path.join(PATH_TO_SAVE_MAIN, 'spline_quadratic_free', 'chebyshev')},
  'spline_quadratic_clamped': {'linear': os.path.join(PATH_TO_SAVE_MAIN, 'spline_quadratic_clamped', 'linear'),
                               'chebyshev': os.path.join(PATH_TO_SAVE_MAIN, 'spline_quadratic_clamped', 'chebyshev')},
  'spline_cubic_free': {'linear': os.path.join(PATH_TO_SAVE_MAIN, 'spline_cubic_free', 'linear'),
                        'chebyshev': os.path.join(PATH_TO_SAVE_MAIN, 'spline_cubic_free', 'chebyshev')},
  'spline_cubic_notaknot': {'linear': os.path.join(PATH_TO_SAVE_MAIN, 'spline_cubic_notaknot', 'linear'),
                            'chebyshev': os.path.join(PATH_TO_SAVE_MAIN, 'spline_cubic_notaknot', 'chebyshev')}
}

# Create directories
for interp_type in paths:
  for node_type in paths[interp_type]:
    os.makedirs(paths[interp_type][node_type], exist_ok=True)

# Initialize list for CSV data
data_records = []

# Generate x-value
plot_x = np.linspace(interval[0], interval[1], 1000)
y_lim_plot = (-10, 40)
x_lim_plot = (-6,6)


for i in N_SET:
  print(f"Processing {i} nodes...")
  # generate_function_uniform_nodes and generate_function_chebyshev_nodes
  # already return Nx2 arrays, which is the correct format for the `nodes` argument
  nodes_linear = generate_function_uniform_nodes(i, interval, func)
  nodes_chebyshev = generate_function_chebyshev_nodes(i, interval, func)

  try:
    quad_spline_linear_free = get_quadratic_spline_interpolation(nodes_linear, None, 'free')
    quad_spline_chebyshev_free = get_quadratic_spline_interpolation(nodes_chebyshev, None, 'free')
    quad_spline_linear_clamped = get_quadratic_spline_interpolation(nodes_linear, func_second_deriv, 'clamped')
    quad_spline_chebyshev_clamped = get_quadratic_spline_interpolation(nodes_chebyshev, func_second_deriv, 'clamped')
    cubic_spline_linear_free = get_cubic_spline_interpolation(nodes_linear, 'free')
    cubic_spline_chebyshev_free = get_cubic_spline_interpolation(nodes_chebyshev, 'free')
    cubic_spline_linear_notaknot = get_cubic_spline_interpolation(nodes_linear, 'not-a-knot')
    cubic_spline_chebyshev_notaknot = get_cubic_spline_interpolation(nodes_chebyshev, 'not-a-knot')
  except Exception as e:
    print(f"Error during interpolation calculation for i={i}: {e}")
    continue # Skip if fails

  interpolations = {
    'spline_quadratic_free': {'linear': quad_spline_linear_free, 'chebyshev': quad_spline_chebyshev_free},
    'spline_quadratic_clamped': {'linear': quad_spline_linear_clamped, 'chebyshev': quad_spline_chebyshev_clamped},
    'spline_cubic_free': {'linear': cubic_spline_linear_free, 'chebyshev': cubic_spline_chebyshev_free},
    'spline_cubic_notaknot': {'linear': cubic_spline_linear_notaknot, 'chebyshev': cubic_spline_chebyshev_notaknot}
  }
  nodes_map = {'linear': nodes_linear, 'chebyshev': nodes_chebyshev}
  node_names = {'linear': 'Równoodległe', 'chebyshev': 'Czebyszewa'}
  interp_names = {
    'spline_quadratic_free': "Splajn Kwadratowy (Free)",
    'spline_quadratic_clamped': "Splajn Kwadratowy (Clamped)",
    'spline_cubic_free': "Splajn Kubiczny (free)",
    'spline_cubic_notaknot': "Splajn Kubiczny (Not-a-Knot)"
  }
  

  #  Plot Setup
  fig_comb = plt.figure(figsize=(12, 7)) # plt.figure returns Figure object
  # Plot original function ONCE on the combined plot
  y_original_plot = np.array([func(x) for x in plot_x])
  plt.plot(plot_x, y_original_plot, label='Funkcja Oryginalna', linestyle=':', color='black', alpha=0.7)
  # Set limits
  if y_lim_plot: 
    plt.ylim(y_lim_plot)

  plot_idx = 0
  for interp_type, node_data in interpolations.items():
    for node_type, interp_func in node_data.items():
      try:
        error_max = get_max_error(func, interp_func, interval)
        error_squared = get_squared_error(func, interp_func, interval)
      except Exception as e:
        print(f"Error calculating error for {interp_type}/{node_type} with i={i}: {e}")
        error_max = np.inf
        error_squared = np.inf

      # Determine colour based on the boundary condition.
      if 'free' in interp_type:
        color = 'blue'
      elif ('notaknot' in interp_type) or ('clamped' in interp_type):
        color = 'red'
      else:
        color = None 

      # plot saving
      fig_single = plt.figure(figsize=(10, 6)) # create new figure for single plot
      plot_title_single = f'{interp_names[interp_type]}, Węzły {node_names[node_type]}, n={i}'

      plot_function(plot_x, interp_func, nodes=nodes_map[node_type],
                    y_lim=y_lim_plot, function_name=plot_title_single, x_lim=x_lim_plot,
                    nodes_name=f'Węzły {node_names[node_type]}', color=color)

      # Plot the original function again on the single plot for comparison
      plt.plot(plot_x, y_original_plot, label='Funkcja Oryginalna', linestyle=':', color='black', alpha=0.7)

      # title, legend etc setup
      plt.title(plot_title_single) 
      plt.legend() 
      if y_lim_plot: 
        plt.ylim(y_lim_plot) 

      save_path_single = paths[interp_type][node_type]
      plt.savefig(os.path.join(save_path_single, f'{interp_names[interp_type]} Węzły {node_names[node_type]} i={i}.png'))
      plt.close(fig_single) 

      # Add line to the combined plot
      plt.figure(fig_comb.number) 
      plot_title_combined = f'{interp_names[interp_type]}, {node_names[node_type]}'
      # Plot interpolation on combined plot - no nodes shown here for clarity
      plot_function(plot_x, interp_func, nodes=None,  # No nodes on combined plot lines
                    function_name=plot_title_combined, x_lim=x_lim_plot,
                    y_lim=y_lim_plot, color=color)
      plot_idx += 1

      # Append data for CSV
      data_records.append({
        'Number of Nodes': i,
        'Node Type': node_names[node_type],
        'Interpolation Type': interp_names[interp_type],
        'Max Error': error_max,
        'RMSE': error_squared 
      })

  # figure finalisatoin
  plt.figure(fig_comb.number) # Ensure active figure is the combined one
  plt.title(f'Porównanie Interpolacji Splajnami dla n={i} węzłów')
  # Increase ncol for legend if needed
  num_legend_items = len(interpolations) * len(nodes_map) + 1 # +1 for original func
  ncol_legend = (num_legend_items + 2) // 3 # Adjust number of columns dynamically (e.g., 3 cols)
  plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=ncol_legend)
  plt.tight_layout(rect=[0, 0.05, 1, 1]) # Adjust layout to make space for legend below
  plt.savefig(os.path.join(PATH_TO_SAVE_MAIN, f'combined_spline_interpolation_n={i}.png'))
  plt.close(fig_comb) # Close the combined plot figure

#  Save Data to CSV 
def format_float(value):
  try:
    num = float(value)
    if not np.isfinite(num):
      # Represent non-finite numbers consistently
      if np.isinf(num):
        return 'inf' if num > 0 else '-inf'
      elif np.isnan(num):
        return 'nan'
      else: # Should not happen with isfinite=False but handle just in case
        return str(value)

    if abs(num) >= 1e6 or (abs(num) < 1e-4 and num != 0.0):  # Use scientific notation
      return f'{num:.6e}'  # 6 digits precision, exponential format
    else:
      return f'{num:.6f}'  # 6 digits precision, fixed-point
  except (ValueError, TypeError):
    return value  # If it's not convertible to float, leave as is

float_columns = ['Max Error', 'RMSE']
df = pd.DataFrame(data_records)

# Ensure the DataFrame has the necessary columns before formatting
if not df.empty and all(col in df.columns for col in float_columns):
  # Format relevant float columns
  for col in float_columns:
    df[col] = df[col].apply(format_float)

# Save to CSV
csv_path = os.path.join(PATH_TO_SAVE_DATA, 'spline_interpolation_errors_with_clamped.csv') # Updated filename
df.to_csv(csv_path, index=False)

print(f"Processing complete for splines. Plots saved in '{PATH_TO_SAVE_MAIN}', CSV saved to '{csv_path}'.")


In [ ]:
# Combined Plot for Cubic and Quadratic Spline Interpolations with Two Boundary Conditions
# (Przykładowo: Splajn kubiczny i kwadratowy z warunkami swobodnymi i obwarowanymi / not-a-knot)

N_SET = [5,7,10,13,20,50,100,1000]

for N in N_SET:
  # Generowanie wartości x i wartości funkcji oryginalnej
  plot_x = np.linspace(interval[0], interval[1], 1000)
  y_original = np.array([func(x) for x in plot_x])
  y_lim_plot = (-10, 40)
  x_lim_plot = (-6, 6)

  # Definicja typów węzłów: 'linear' dla węzłów równomiernych oraz 'chebyshev' dla węzłów Czebyszewa.
  node_types = ['linear', 'chebyshev']
  node_names = {'linear': 'Równoodległe', 'chebyshev': 'Czebyszewa'}

  # === WYKRESY SPLAJNÓW KUBICZNYCH ===
  # Tworzymy wykres zbiorczy z podwykresami dla każdego typu węzłów.
  fig_cubic, axs_cubic = plt.subplots(1, len(node_types), figsize=(14, 7), sharey=True)

  # Kolory dla warunków brzegowych w splajnie kubicznym:
  colors_cubic = {'Free Boundary': 'blue', 'Not-a-Knot': 'red'}

  for idx, node_type in enumerate(node_types):
    # Generowanie węzłów w zależności od typu.
    if node_type == 'linear':
      nodes = generate_function_uniform_nodes(N, interval, func)
    else:
      nodes = generate_function_chebyshev_nodes(N, interval, func)

    # Obliczenie interpolacji splajnów kubicznych przy użyciu warunków:
    spline_free = get_cubic_spline_interpolation(nodes, 'free')
    spline_notaknot = get_cubic_spline_interpolation(nodes, 'not-a-knot')

    # Rysujemy funkcję oryginalną.
    axs_cubic[idx].plot(plot_x, y_original, label='Funkcja oryginalna',
                         linestyle=':', color='black', alpha=0.7)
    # Rysujemy splajn sześcienny z warunkami swobodnymi.
    y_free = [spline_free(x) for x in plot_x]
    axs_cubic[idx].plot(plot_x, y_free, label='Splajn Sześcienny (Free Boundary)',
                         color=colors_cubic['Free Boundary'])
    # Rysujemy splajn sześcienny z warunkiem not-a-knot.
    y_notaknot = [spline_notaknot(x) for x in plot_x]
    axs_cubic[idx].plot(plot_x, y_notaknot, label='Splajn Sześcienny (Not-a-Knot)',
                         color=colors_cubic['Not-a-Knot'])
    # Rysujemy węzły interpolacji.
    axs_cubic[idx].scatter(nodes[:, 0], nodes[:, 1], color='magenta', marker='o', s=50,
                           label='Węzły interpolacji')

    # Formatowanie wykresu.
    axs_cubic[idx].set_title(f'Splajn Sześcienny – węzły {node_names[node_type]} (n={N})')
    axs_cubic[idx].set_xlim(x_lim_plot)
    axs_cubic[idx].set_ylim(y_lim_plot)
    axs_cubic[idx].grid(True)
    axs_cubic[idx].legend()

  plt.tight_layout()
  # Tworzymy folder dla wykresów splajnów kubicznych, jeśli nie istnieje.
  combined_folder_cubic = os.path.join(PATH_TO_SAVE_MAIN, 'combined_cubic_spline_comparison')
  os.makedirs(combined_folder_cubic, exist_ok=True)
  save_path_cubic = os.path.join(combined_folder_cubic, f'combined_cubic_spline_interpolation_n={N}.png')
  plt.savefig(save_path_cubic)
  plt.show()
  print(f"Wykresy splajnów sześciennych zapisane w: {save_path_cubic}")

  # === WYKRESY SPLAJNÓW KWADRATOWYCH ===
  # Tworzymy wykres zbiorczy z podwykresami dla każdego typu węzłów.
  fig_quad, axs_quad = plt.subplots(1, len(node_types), figsize=(14, 7), sharey=True)

  # Kolory dla warunków brzegowych w splajnie kwadratowym:
  colors_quadratic = {'Free Boundary': 'blue', 'Clamped': 'red'}

  for idx, node_type in enumerate(node_types):
    # Generowanie węzłów w zależności od typu.
    if node_type == 'linear':
      nodes = generate_function_uniform_nodes(N, interval, func)
    else:
      nodes = generate_function_chebyshev_nodes(N, interval, func)

    # Obliczenie interpolacji splajnów kwadratowych przy użyciu warunków:
    # Dla warunku swobodnego przekazujemy None jako pochodną.
    spline_free = get_quadratic_spline_interpolation(nodes, None, 'free')
    spline_clamped = get_quadratic_spline_interpolation(nodes, func_second_deriv, 'clamped')

    # Rysujemy funkcję oryginalną.
    axs_quad[idx].plot(plot_x, y_original, label='Funkcja oryginalna',
                        linestyle=':', color='black', alpha=0.7)
    # Rysujemy splajn kwadratowy z warunkami swobodnymi.
    y_free = [spline_free(x) for x in plot_x]
    axs_quad[idx].plot(plot_x, y_free, label='Splajn Kwadratowy (Free Boundary)',
                        color=colors_quadratic['Free Boundary'])
    # Rysujemy splajn kwadratowy z warunkami obwarowanymi.
    y_clamped = [spline_clamped(x) for x in plot_x]
    axs_quad[idx].plot(plot_x, y_clamped, label='Splajn Kwadratowy (Clamped Boundary)',
                        color=colors_quadratic['Clamped'])
    # Rysujemy węzły interpolacji.
    axs_quad[idx].scatter(nodes[:, 0], nodes[:, 1], color='magenta', marker='o', s=50,
                          label='Węzły interpolacji')

    # Formatowanie wykresu.
    axs_quad[idx].set_title(f'Splajn Kwadratowy – węzły {node_names[node_type]} (n={N})')
    axs_quad[idx].set_xlim(x_lim_plot)
    axs_quad[idx].set_ylim(y_lim_plot)
    axs_quad[idx].grid(True)
    axs_quad[idx].legend()

  plt.tight_layout()
  # Tworzymy folder dla wykresów splajnów kwadratowych, jeśli nie istnieje.
  combined_folder_quad = os.path.join(PATH_TO_SAVE_MAIN, 'combined_quadratic_spline_comparison')
  os.makedirs(combined_folder_quad, exist_ok=True)
  save_path_quad = os.path.join(combined_folder_quad, f'combined_quadratic_spline_interpolation_n={N}.png')
  plt.savefig(save_path_quad)
  plt.show()
  print(f"Wykresy splajnów kwadratowych zapisane w: {save_path_quad}")


In [ ]:
# Combined Plot for Cubic and Quadratic Spline Interpolations with Two Boundary Conditions
N_SET = [5,7,10,13,20,50,100,1000]

for N in N_SET:
  # Generowanie wartości x i wartości funkcji oryginalnej
  plot_x = np.linspace(interval[0], interval[1], 1000)
  y_original = np.array([func(x) for x in plot_x])
  y_lim_plot = (-10, 40)
  x_lim_plot = (-6, 6)

  # Definicja typów węzłów: 'linear' dla węzłów równomiernych oraz 'chebyshev' dla węzłów Czebyszewa.
  node_types = ['linear', 'chebyshev']
  node_names = {'linear': 'Równoodległe', 'chebyshev': 'Czebyszewa'}

  # === WYKRESY SPLAJNÓW KUBICZNYCH ===
  # Tworzymy wykres zbiorczy z podwykresami dla każdego typu węzłów.
  fig_cubic, axs_cubic = plt.subplots(1, len(node_types), figsize=(14, 7), sharey=True)

  # Kolory dla warunków brzegowych w splajnie kubicznym:
  colors_cubic = {'Free Boundary': 'blue', 'Not-a-Knot': 'red'}

  for idx, node_type in enumerate(node_types):
    # Generowanie węzłów w zależności od typu.
    if node_type == 'linear':
      nodes = generate_function_uniform_nodes(N, interval, func)
    else:
      nodes = generate_function_chebyshev_nodes(N, interval, func)

    # Obliczenie interpolacji splajnów kubicznych przy użyciu warunków:
    spline_free = get_cubic_spline_interpolation(nodes, 'free')
    spline_notaknot = get_cubic_spline_interpolation(nodes, 'not-a-knot')

    # Rysujemy funkcję oryginalną.
    axs_cubic[idx].plot(plot_x, y_original, label='Funkcja oryginalna',
                         linestyle=':', color='black', alpha=0.7)
    # Rysujemy splajn kubiczny z warunkami swobodnymi.
    y_free = [spline_free(x) for x in plot_x]
    axs_cubic[idx].plot(plot_x, y_free, label='Splajn Sześcienny (Free Boundary)',
                         color=colors_cubic['Free Boundary'])
    # Rysujemy splajn Sześcienny z warunkiem not-a-knot.
    y_notaknot = [spline_notaknot(x) for x in plot_x]
    axs_cubic[idx].plot(plot_x, y_notaknot, label='Splajn Sześcienny (Not-a-Knot)',
                         color=colors_cubic['Not-a-Knot'])

    # Formatowanie wykresu.
    axs_cubic[idx].set_title(f'Splajn Sześcienny – węzły {node_names[node_type]} (n={N})')
    axs_cubic[idx].set_xlim(x_lim_plot)
    axs_cubic[idx].set_ylim(y_lim_plot)
    axs_cubic[idx].grid(True)
    axs_cubic[idx].legend()

  plt.tight_layout()
  # Tworzymy folder dla wykresów splajnów kubicznych, jeśli nie istnieje.
  combined_folder_cubic = os.path.join(PATH_TO_SAVE_MAIN, 'combined_cubic_spline_comparison')
  os.makedirs(combined_folder_cubic, exist_ok=True)
  save_path_cubic = os.path.join(combined_folder_cubic, f'combined_cubic_spline_interpolation_n={N}.png')
  plt.savefig(save_path_cubic)
  plt.show()
  print(f"Wykresy splajnów sześciennych zapisane w: {save_path_cubic}")

  # === WYKRESY SPLAJNÓW KWADRATOWYCH ===
  # Tworzymy wykres zbiorczy z podwykresami dla każdego typu węzłów.
  fig_quad, axs_quad = plt.subplots(1, len(node_types), figsize=(14, 7), sharey=True)

  # Kolory dla warunków brzegowych w splajnie kwadratowym:
  colors_quadratic = {'Free Boundary': 'blue', 'Clamped': 'red'}

  for idx, node_type in enumerate(node_types):
    # Generowanie węzłów w zależności od typu.
    if node_type == 'linear':
      nodes = generate_function_uniform_nodes(N, interval, func)
    else:
      nodes = generate_function_chebyshev_nodes(N, interval, func)

    # Obliczenie interpolacji splajnów kwadratowych przy użyciu warunków:
    # Dla warunku swobodnego przekazujemy None jako pochodną.
    spline_free = get_quadratic_spline_interpolation(nodes, None, 'free')
    spline_clamped = get_quadratic_spline_interpolation(nodes, func_second_deriv, 'clamped')

    # Rysujemy funkcję oryginalną.
    axs_quad[idx].plot(plot_x, y_original, label='Funkcja oryginalna',
                        linestyle=':', color='black', alpha=0.7)
    # Rysujemy splajn kwadratowy z warunkami swobodnymi.
    y_free = [spline_free(x) for x in plot_x]
    axs_quad[idx].plot(plot_x, y_free, label='Splajn Kwadratowy (Free Boundary)',
                        color=colors_quadratic['Free Boundary'])
    # Rysujemy splajn kwadratowy z warunkami obwarowanymi.
    y_clamped = [spline_clamped(x) for x in plot_x]
    axs_quad[idx].plot(plot_x, y_clamped, label='Splajn Kwadratowy (Clamped Boundary)',
                        color=colors_quadratic['Clamped'])

    # Formatowanie wykresu.
    axs_quad[idx].set_title(f'Splajn Kwadratowy – węzły {node_names[node_type]} (n={N})')
    axs_quad[idx].set_xlim(x_lim_plot)
    axs_quad[idx].set_ylim(y_lim_plot)
    axs_quad[idx].grid(True)
    axs_quad[idx].legend()

  plt.tight_layout()
  # Tworzymy folder dla wykresów splajnów kwadratowych, jeśli nie istnieje.
  combined_folder_quad = os.path.join(PATH_TO_SAVE_MAIN, 'combined_quadratic_spline_comparison')
  os.makedirs(combined_folder_quad, exist_ok=True)
  save_path_quad = os.path.join(combined_folder_quad, f'combined_quadratic_spline_interpolation_n={N}.png')
  plt.savefig(save_path_quad)
  plt.show()
  print(f"Wykresy splajnów kwadratowych zapisane w: {save_path_quad}")
